In [1]:
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=3fdc103e88dd39d2b0bd6f1a8475a93bc9f3ad58c6ce3fde973f95fdf3e5ce49
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer  # Corrected import for Aer simulator
from qiskit.circuit.library import QFT
import numpy as np

# Use Aer simulator backend
backend = Aer.get_backend('qasm_simulator')


In [7]:

# PART 1
# Create the quantum circuit with 3 qubits and 3 classical bits
q = QuantumRegister(3, 'q')  # Quantum register
c0 = ClassicalRegister(1, 'c0')  # Classical register for Alice's qubit
c1 = ClassicalRegister(1, 'c1')  # Classical register for Bob's qubit
c2 = ClassicalRegister(1, 'c2')  # Classical register for the result
circuit = QuantumCircuit(q, c0, c1, c2)

# Prepare the initial state to be teleported
circuit.initialize([0, 1], q[0])  # Initialize q[0] in the |1> state
circuit.barrier()

# Create an entanglement between Alice's and Bob's qubits
circuit.h(q[1])  # Hadamard on Bob's qubit
circuit.cx(q[1], q[2])  # CNOT between Bob's qubits
circuit.barrier()

# Teleportation process
circuit.cx(q[0], q[1])  # CNOT for teleportation
circuit.h(q[0])  # Hadamard on Alice's qubit
circuit.barrier()

# Measure Alice's qubits and send the measurement results to Bob
circuit.measure(q[0], c0[0])  # Measure Alice's qubit
circuit.measure(q[1], c1[0])  # Measure Bob's qubit

# Apply corrective operations on Bob's qubit based on the measurement results
circuit.x(q[2]).c_if(c1, 1)  # Apply X gate if Bob's measurement is 1
circuit.z(q[2]).c_if(c0, 1)  # Apply Z gate if Alice's measurement is 1

# Measure the teleported qubit
circuit.measure(q[2], c2[0])  # Measure the teleported state

# Visualize the circuit
print(circuit)
circuit.draw(output='mpl')

# Simulate the circuit using the QASM simulator
transpiled_circuit = transpile(circuit, backend)  # Transpile the circuit for the simulator
job = backend.run(transpiled_circuit, shots=1)  # Run the transpiled circuit
result = job.result()
teleported_state = result.get_counts(transpiled_circuit)  # Get counts from the transpiled circuit

# Print the teleported state
print("Teleported state:", teleported_state)

      ┌─────────────────┐ ░            ░      ┌───┐ ░ ┌─┐                    
 q_0: ┤ Initialize(0,1) ├─░────────────░───■──┤ H ├─░─┤M├────────────────────
      └─────────────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐                 
 q_1: ────────────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─────────────────
                          ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ┌───┐  ┌───┐ ┌─┐
 q_2: ────────────────────░──────┤ X ├─░────────────░──╫──╫──┤ X ├──┤ Z ├─┤M├
                          ░      └───┘ ░            ░  ║  ║  └─╥─┘  └─╥─┘ └╥┘
                                                       ║  ║    ║   ┌──╨──┐ ║ 
c0: 1/═════════════════════════════════════════════════╩══╬════╬═══╡ 0x1 ╞═╬═
                                                       0  ║ ┌──╨──┐└─────┘ ║ 
c1: 1/════════════════════════════════════════════════════╩═╡ 0x1 ╞════════╬═
                                                          0 └─────┘        ║ 
c2: 1/══════════════════════════════════════════════════════════

In [11]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer  # Corrected import for Aer simulator

from qiskit.circuit.library import QFT

import numpy as np



# Initialize quantum and classical registers
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

# Create the quantum circuit
circuit = QuantumCircuit(q, c)

# Prepare the circuit
circuit.x(q[4])  # Apply X gate to q[4]
circuit.x(q[2])  # Apply X gate to q[2]
circuit.x(q[0])  # Apply X gate to q[0]
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=False, insert_barriers=False), q)
circuit.measure(q, c)  # Measure the qubits
circuit.draw(output='mpl', filename='qft1.png')  # Draw the circuit and save to a file
print(circuit)

# Transpile the circuit for the backend
transpiled_circuit = transpile(circuit, backend)

# Run the job
job = backend.run(transpiled_circuit, shots=1000)

# Get the results
result = job.result()
counts = result.get_counts()

# Display the QFT output
print("\nQFT Output")
print("-------------")
print(counts)



     ┌───┐┌──────┐┌─┐            
q_0: ┤ X ├┤0     ├┤M├────────────
     └───┘│      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 QFT ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
     ┌───┐│      │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 

QFT Output
-------------
{'10101': 24, '01111': 26, '10110': 31, '01101': 21, '11110': 36, '01000': 37, '01100': 26, '01001': 37, '10001': 25, '00101': 34, '11010': 27, '10000': 34, '11100': 45, '10010': 39, '00110': 39, '00001': 22, '00010': 30, '01010': 24, '11001': 31, '00000': 33, '11011': 30, '11111': 38, '11000': 23, '00011': 38, '00111': 27, '10111': 38, '10100': 40, '11101': 23, '10011': 33, '01011': 28, '00100': 30, '01110': 31}


In [15]:
# PART 3
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

circuit = QuantumCircuit(q, c)

circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=False, insert_barriers=False, name='qft'), q)
circuit.measure(q, c)
circuit.draw(output='mpl', filename='qft1.png')
print(circuit)

# Transpile the circuit for the backend
transpiled_circuit = transpile(circuit, backend)

# Run the job
job = backend.run(transpiled_circuit, shots=1000)

# Get the results
result = job.result()
counts = result.get_counts()

# Display the QFT output
print("\nQFT Output")
print("-------------")
print(counts)


# Create another circuit for QFT with Inverse
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

circuit = QuantumCircuit(q, c)

circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=True, insert_barriers=False, name='qft_inv'), q)
circuit.measure(q, c)
circuit.draw(output='mpl', filename='qft2.png')

print(circuit)

# Transpile the circuit for the backend
transpiled_circuit = transpile(circuit, backend)

# Run the job
job = backend.run(transpiled_circuit, shots=1000)

# Get the results
result = job.result()
counts = result.get_counts()

# Display the QFT output
print("\n QFT with inverse QFT Output")
print("-------------")
print(counts)



     ┌───┐┌──────┐┌─┐            
q_0: ┤ X ├┤0     ├┤M├────────────
     └───┘│      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 qft ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
     ┌───┐│      │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 

QFT Output
-------------
{'00101': 21, '01101': 26, '00111': 26, '00011': 27, '01011': 32, '11101': 36, '00010': 27, '10100': 35, '10111': 38, '11100': 30, '10101': 36, '10010': 28, '01010': 23, '00000': 25, '11001': 22, '01111': 35, '10110': 35, '01110': 24, '01100': 34, '01000': 37, '00001': 37, '00110': 30, '11000': 29, '11110': 38, '10000': 36, '11111': 33, '11011': 29, '11010': 30, '01001': 33, '10001': 34, '10011': 41, '00100': 33}
     ┌───┐┌──────────┐┌─┐            
q_0: ┤ X ├┤0         ├┤M├────────────
     └───┘│          │└╥┘┌─┐         
q